In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
import decimal
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm
from sklearn.svm import SVR

#data = pd.read_csv('data', parse_dates=[[0,1]], infer_datetime_format=True)
df = pd.read_csv("training.csv", infer_datetime_format=True,low_memory=False)

# Remove the following columns since it has too many missing values
df = df.drop(['X16','X25','X26'], axis=1)
# Remove NaN values if any of the column has a NaN
#df = df.dropna()


# X1 is the response column, thus removing all NaNs from it
# X4,X5,X6 are currency and need to be converted to int
# Pandas can not store NaN in int type column, thus had remove all NaNs from them as well
for col in ('X1','X4', 'X5','X6'):
    df = df[pd.notnull(df[col])]

D = decimal.Decimal
# Pandas can not store NaN in int type column, thus had remove all NaNs
for col in ('X4', 'X5','X6'):
    df[col] = df[col].str.replace(r'[^-+\d.]', '').astype(D)
    df[col] = df[col].astype('int')
    
# Convert columns having % from obj(string type) to float and divide by 100 
for col in ('X1', 'X30'):
    df[col] = df[col].str.replace('%','').astype('float')/100

#df.dtypes

In [111]:
#df = pd.read_csv("training.csv", parse_dates=True,index_col = [14,22],infer_datetime_format=True,low_memory=False)
## Need to deal with date, maybe convert it to month or days of the year,since interest rate is related to timing

In [54]:
#df.describe()
#df.ix[:, 10:23].describe()

# Column X32,X7,X8,X24,X14,X12,X17,X22 unique values are very few, need to look into further if they are sparse predictors
# regression based models need to remove sparse predictors

# Column X26,X25,X16 a lot of missing values, maybe worth removing them

,X13,X21,X22
count,338972.000000,399999.000000,399999.000000
mean,73160.149695,17.000786,0.274528
std,55867.696483,7.704189,0.778209
min,3000.000000,0.000000,0.000000
25%,45000.000000,11.250000,0.000000
50%,63000.000000,16.700000,0.000000
75%,88200.000000,22.520000,0.000000
max,7500000.000000,39.990000,29.000000


In [34]:
# Define X and Y vectors for model fitting
# flattern the vector column y into an 1-d array which is a required format for model fitting
y = np.ravel(df.ix[:, :1])
X = df.ix[:, 1:5]

In [35]:
# n_samples, n_features = 10, 5
# np.random.seed(0)
# y = np.random.randn(n_samples)
# X = np.random.randn(n_samples, n_features)
from sklearn import linear_model

clf = linear_model.SGDRegressor()
clf.fit(X, y)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [ ]:
# Try random forest
#import randomforests as rf
# rf super slow, and interation 10 is not sufficient
# with error thrown out ome inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
#  warn("Some inputs do not have OOB scores. "
import sklearn.ensemble as sk
rfc = sk.RandomForestClassifier(n_estimators=100, oob_score=True)
model = rfc.fit(X, y)

In [36]:
## Try Support Vector Regression
# failed as there are a lot string type predictors in Y
clf = SVR(C=1.0, epsilon=0.2)
clf.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma=0.0,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [40]:
## Odinary Linear Regression
model = sm.OLS(y,X)
f = model.fit()

print ('Coefficients: ', f.params[0:2])
print ('Intercept: ', f.params[2])
print ('P-Values: ', f.pvalues)
print ('R-Squared: ', f.rsquared)

Coefficients:  X2   -1.140427e-08
X3    1.357400e-08
dtype: float64
Intercept:  6.255497628e-06
P-Values:  X2     0.000000e+00
X3     0.000000e+00
X4    5.828193e-142
X5     4.125758e-04
dtype: float64
R-Squared:  0.781409902785


In [41]:
#http://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html
## Gradient_boosting
from sklearn import ensemble

# Fit regression model
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 1,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X, y)
#mse = mean_squared_error(y_test, clf.predict(X_test))
#print("MSE: %.4f" % mse)

# Plot feature importance
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, boston.feature_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.01, loss='ls',
             max_depth=4, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=1,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

In [44]:
#http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html
# Decision tree regression with adaboost
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
rng = np.random.RandomState(1)
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)
regr_1.fit(X, y)
regr_2.fit(X, y)

#http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=None,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x0000000010974EB8>)